In [1]:
%%time
%%capture

!pip install langchain
!pip install langchain-openai
!pip install faiss-gpu
!pip install sentence_transformers
!pip install openai
!pip install python-dotenv

CPU times: user 48.7 ms, sys: 8 ms, total: 56.7 ms
Wall time: 9.77 s


In [8]:
%%time
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain_text_splitters import CharacterTextSplitter
from openai import OpenAI
import os
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_openai import ChatOpenAI
from langchain_community.document_loaders import TextLoader

from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain
from langchain_core.vectorstores import VectorStoreRetriever
from langchain.chains import RetrievalQA

from langchain_core.vectorstores import VectorStoreRetriever
from langchain.chains import RetrievalQA

from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

%load_ext dotenv
%dotenv

token = os.environ.get('ONE_API_TOKEN')
base_url=os.environ.get('ONE_API_URL')

CPU times: user 44.8 ms, sys: 1.76 ms, total: 46.6 ms
Wall time: 45.9 ms


In [2]:
%%time

model_name='/models/bge-m3'
model_kwargs = {"device": "cuda"}
encode_kwargs = {"normalize_embeddings": True}
embeddings = HuggingFaceBgeEmbeddings(
    model_name=model_name, model_kwargs=model_kwargs, encode_kwargs=encode_kwargs
)

CPU times: user 3.01 s, sys: 1.05 s, total: 4.06 s
Wall time: 3.17 s


In [3]:
%%time
%%capture

documents=[]
loader = TextLoader("./孔乙己.txt")
documents.extend(loader.load())

CPU times: user 820 µs, sys: 0 ns, total: 820 µs
Wall time: 834 µs


In [4]:
%%time

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter=RecursiveCharacterTextSplitter(chunk_size=128,chunk_overlap=20)
text_splits=text_splitter.split_documents(documents)

len(text_splits)

CPU times: user 4.62 ms, sys: 0 ns, total: 4.62 ms
Wall time: 4.31 ms


39

In [7]:
%%time

vectorstore = FAISS.from_documents(text_splits, embeddings)

CPU times: user 758 ms, sys: 60.5 ms, total: 819 ms
Wall time: 816 ms


In [6]:
%%time
chat = ChatOpenAI(api_key=token, 
                  base_url=base_url,
                  model='xiaoyu', 
                  temperature=0,
                  streaming=True)

CPU times: user 91.6 ms, sys: 1.76 ms, total: 93.3 ms
Wall time: 92.1 ms


In [22]:
%%time

from langchain_core.vectorstores import VectorStoreRetriever
from langchain.chains import RetrievalQA


retriever = VectorStoreRetriever(vectorstore=vectorstore,search_kwargs={"k": 10})

qa_chain = RetrievalQA.from_chain_type(
    llm=chat, retriever=retriever
)

CPU times: user 297 µs, sys: 0 ns, total: 297 µs
Wall time: 301 µs


In [24]:
%%time

from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain

prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}""")

document_chain = create_stuff_documents_chain(chat, prompt)

CPU times: user 422 µs, sys: 0 ns, total: 422 µs
Wall time: 427 µs


In [25]:
%%time

retriever = vectorstore.as_retriever(search_kwargs={"k": 10,"score_threshold": 0.4},search_type="similarity_score_threshold")
retrieval_chain = create_retrieval_chain(retriever, document_chain)

CPU times: user 473 µs, sys: 0 ns, total: 473 µs
Wall time: 477 µs


In [27]:
%%time

from bge_rerank import BgeRerank
from langchain.chains import RetrievalQA
from langchain.retrievers import ContextualCompressionRetriever

compressor = BgeRerank()

CPU times: user 1.49 s, sys: 555 ms, total: 2.04 s
Wall time: 1.09 s


In [28]:
%%time

retrieval_chain = create_retrieval_chain(compressor, retrieval_chain)

AttributeError: 'BgeRerank' object has no attribute 'with_config'